In [3]:
import os
import re
from parse import search,findall
from pathlib import Path
path = Path(".")

import numpy as np
import pandas as pd 

In [4]:
total = 0
for file in path.glob("*.txt"):
    with open(file,'r') as f:
        n_game = 0
        for line in f:
            if "Game ended at" in line :
                n_game += 1
        total += n_game
        display ("file " + file.name + " have " + str(n_game))
display ("Total game is " + str(total))

'file Export Holdem Manager 2.0 12292016131233.txt have 2468'

'file Export Holdem Manager 2.0 12302016144830.txt have 42688'

'file File196.txt have 1000'

'file File198.txt have 1000'

'file File199.txt have 1000'

'file File200.txt have 1000'

'file File201.txt have 1000'

'file File203.txt have 1000'

'file File204.txt have 1000'

'Total game is 52156'

In [5]:
file1 = Path('File196.txt')

In [6]:
with file1.open() as f:
    lines = f.readlines()

In [7]:
lines = [line.strip() for line in lines]

In [8]:
games = [] 
y = 0
for n_line, line in enumerate(lines):
    if "Game ended at" in line :
        games.append(lines[y:n_line+1])
        y = n_line+4



In [9]:
games[1]

['Game started at: 2016/9/4 1:7:4',
 "Game ID: 718895155 2/4 (PRR) Medusa (Short) (Hold'em)",
 'Seat 5 is the button',
 'Seat 4: bjv1105 (200).',
 'Seat 5: IlxxxlI (88).',
 'Player IlxxxlI has small blind (2)',
 'Player bjv1105 has big blind (4)',
 'Player bjv1105 received a card.',
 'Player bjv1105 received a card.',
 'Player IlxxxlI received card: [8c]',
 'Player IlxxxlI received card: [8s]',
 'Player IlxxxlI allin (86)',
 'Player bjv1105 folds',
 'Uncalled bet (84) returned to IlxxxlI',
 'Player IlxxxlI mucks cards',
 '------ Summary ------',
 'Pot: 8. Rake 0',
 'Player bjv1105 does not show cards.Bets: 4. Collects: 0. Loses: 4.',
 '*Player IlxxxlI mucks (does not show cards). Bets: 4. Collects: 8. Wins: 4.',
 'Game ended at: 2016/9/4 1:7:11']

In [101]:
class Game:
    @classmethod
    def read_gameinfo(cls,gameinfo):
        hasRiver = False
        hasTurn = False
        hasFlop = False
        hasShowdown = False
        tmp = []
        loser = []
        loser_lost = []
        winner = []
        profit = []
        river_card = []
        self_card= []
        turn_card = []
        flop_card = []
        count = 0           
        for line in gameinfo:
            if " is the button" in line:
                button = int(line.split()[1])
            elif "Player IlxxxlI received card:" in line:
                self_card = self_card + [r.fixed[0] for r in findall("[{}]", line)]
            elif "*Player" in line :
                winner.append(line.split(" ")[1])
                profit.append(float(re.findall("Wins: (\d+(\.\d+)?)",line)[0][0]))
            elif "*** FLOP ***" in line :
                hasFlop = True
                flop_card = [r.fixed[0] for r in findall("[{}]", line)]
                flop_card = flop_card[0].split(" ")
            elif "*** TURN ***" in line :
                hasTurn = True
                turn_card = [r.fixed[0] for r in findall("[{}]", line)]
                turn_card = turn_card[0].split(" ") + [turn_card[1]]
            elif "*** RIVER ***" in line :
                hasRiver = True
                river_card = [r.fixed[0] for r in findall("[{}]", line)]
                river_card = river_card[0].split(" ") + [river_card[1]]
                count = 1;
                hasShowdown = True
            elif "Pot: " in line:
                tmp = re.findall("(\d+(\.\d+)?)",line)
                pot = float(tmp[0][0])
                rake = float(tmp[1][0])
                jp_fee = float(tmp[2][0])
            elif "------ Summary ------" in line :
                count = -99
            elif count == 1 :
                if " Uncalled bet " in line :
                    hasShowdown = False
                    count = count -1
            elif count==-99 :
                if "Loses:" in line :
                    loser.append(line.split(" ")[1])
                    loser_lost.append(float(re.findall("Loses: (\d+(\.\d+)?)",line)[0][0]))
                
                
        lines_entire = '\n'.join(gameinfo)
        seat_info = [r.named for r in findall("Seat {num:d}: {user:w} ({stack}).", lines_entire)]
        
        return cls(winner,profit,loser,loser_lost,button,seat_info,pot,rake,jp_fee,self_card,hasFlop,flop_card,hasTurn,turn_card,hasRiver,river_card,hasShowdown)
    def __init__(self,winner,profit,loser,loser_lost,button,seat_info,pot,rake,jp_fee,self_card,hasFlop = False,flop_card =[],hasTurn = False,turn_card =[],hasRiver = False,river_card = [],hasShowdown = False):
        self.winner = winner
        self.profit = profit
        self.loser = loser
        self.loser_lost = loser_lost
        self.button = button
        self.seat_info = seat_info
        self.No_player = len(seat_info) / 11
        self.pot = pot
        self.rake = rake
        self.jp_fee = jp_fee
        self.self_card = self_card
        self.hasFlop = hasFlop
        self.flop_card = flop_card
        self.hasTurn = hasTurn
        self.turn_card = turn_card
        self.hasRiver = hasRiver
        self.river_card = river_card
        self.hasShowdown = hasShowdown 
        for x in range(0,len(seat_info)):
            if seat_info[x]['user'] == winner[0] :
                self.ratio = profit[0] / float(seat_info[x]['stack'])
                for y in range(0,len(seat_info)):
                    if x+3+y > len(seat_info)-1 :
                        x= x-len(seat_info)
                    seat_info[x+3+y]['position'] = y+1
                    seat_info[x+3+y]['position_ratio'] = (y+1) / len(seat_info)


                
        
        
        
        
        

In [102]:
game = Game.read_gameinfo(games[23])
game.seat_info

[{'num': 1,
  'user': 'BIGRAISE',
  'stack': '120',
  'position': 8,
  'position_ratio': 0.8888888888888888},
 {'num': 2,
  'user': 'tcoll16',
  'stack': '229.62',
  'position': 9,
  'position_ratio': 1.0},
 {'num': 3,
  'user': 'cracypoker',
  'stack': '261.95',
  'position': 1,
  'position_ratio': 0.1111111111111111},
 {'num': 4,
  'user': 'VegetablesArentYummy',
  'stack': '200',
  'position': 2,
  'position_ratio': 0.2222222222222222},
 {'num': 5,
  'user': 'Oakee',
  'stack': '200',
  'position': 3,
  'position_ratio': 0.3333333333333333},
 {'num': 6,
  'user': 'IlxxxlI',
  'stack': '80',
  'position': 4,
  'position_ratio': 0.4444444444444444},
 {'num': 7,
  'user': 'WalterBlack',
  'stack': '475.85',
  'position': 5,
  'position_ratio': 0.5555555555555556},
 {'num': 8,
  'user': 'TheFront7',
  'stack': '299.20',
  'position': 6,
  'position_ratio': 0.6666666666666666},
 {'num': 9,
  'user': 'NoSugarJoe',
  'stack': '16.20',
  'position': 7,
  'position_ratio': 0.7777777777777778

In [12]:
games[23]

['Game started at: 2016/9/4 2:15:42',
 "Game ID: 718949125 2/4 (PRR) Monopod (Short) - 2 (Hold'em)",
 'Seat 9 is the button',
 'Seat 1: BIGRAISE (120).',
 'Seat 2: tcoll16 (229.62).',
 'Seat 3: cracypoker (261.95).',
 'Seat 4: VegetablesArentYummy (200).',
 'Seat 5: Oakee (200).',
 'Seat 6: IlxxxlI (80).',
 'Seat 7: WalterBlack (475.85).',
 'Seat 8: TheFront7 (299.20).',
 'Seat 9: NoSugarJoe (16.20).',
 'Player BIGRAISE has small blind (2)',
 'Player tcoll16 has big blind (4)',
 'Player BIGRAISE received a card.',
 'Player BIGRAISE received a card.',
 'Player tcoll16 received a card.',
 'Player tcoll16 received a card.',
 'Player cracypoker received a card.',
 'Player cracypoker received a card.',
 'Player VegetablesArentYummy received a card.',
 'Player VegetablesArentYummy received a card.',
 'Player Oakee received a card.',
 'Player Oakee received a card.',
 'Player IlxxxlI received card: [5d]',
 'Player IlxxxlI received card: [Qc]',
 'Player WalterBlack received a card.',
 'Player 

In [36]:
5/222

0.02252252252252252